In [1]:
import torchvision
import torch
import torchvision.transforms as transforms
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
DATA_PATH = r"C:\Users\offco\Documents\Dev_Projects\BookCoverClassifier\Datasets\Test_Run\Test_Augment"

mean = [0.3979, 0.3628, 0.3351]
std = [0.3045, 0.2798, 0.2624]

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor(mean), torch.Tensor(std)),
])

data = torchvision.datasets.ImageFolder(root=DATA_PATH, transform=transform)

train_size = int(0.85 * len(data))
test_size = len(data) - train_size
train_data, test_data = torch.utils.data.random_split(data, [train_size, test_size])

In [3]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

In [4]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

In [5]:
def set_device():
    if torch.cuda.is_available(): dev = 'cuda:0'
    else: dev = 'cpu'
    
    return torch.device(dev)

In [6]:
def evaluate_model(model, test_loader):
    model.eval()
    
    predicted_correctly_on_epoch = 0
    total = 0
    
    device = set_device()
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            total += labels.size(0)
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            
            predicted_correctly_on_epoch += (predicted == labels).sum().item()
    
    epoch_acc = 100.0 * (predicted_correctly_on_epoch / total)
    print(f"Testing dataset: {int(predicted_correctly_on_epoch)} out of {total} correct. Accuracy: {round(epoch_acc, 3)}")
    
    return epoch_acc

In [7]:
CHECKPOINT_PATH = r"C:\Users\offco\Documents\Dev_Projects\BookCoverClassifier\Test_Run\Model\PyTorch Model"

def save_checkpoint(model, epoch, optimizer, best_acc):
    state = {
        'epoch': epoch+1,
        'model': model.state_dict(),
        'best accuracy': best_acc,
        'optimizer': optimizer.state_dict()
    }
    
    torch.save(state, f'{CHECKPOINT_PATH}\model_best_checkpoint.pth.tar')

In [8]:
def train_nn(model, train_loader, test_loader, criterion, optimizer, n_epochs):
    device = set_device()
    best_acc = 0
    
    for epoch in range(n_epochs):
        print(f"Epoch: {epoch+1}")
        model.train()
        running_loss = 0.0
        running_correct = 0.0
        total = 0
        
        for data in train_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            total += labels.size(0)
            
            optimizer.zero_grad()
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            
            loss = criterion(outputs, labels)
            loss.backward()
            
            optimizer.step()
            
            running_loss += loss.item()
            running_correct += (labels == predicted).sum().item()
        
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100.00 * (running_correct / total)
        
        print(f"Training dataset: {int(running_correct)} out of {total} correct. Accuracy: {round(epoch_acc, 3)}. Loss: {round(epoch_loss, 3)}")
        
        test_acc = evaluate_model(model, test_loader)
        
        if (test_acc > best_acc):
            best_acc = test_acc
            save_checkpoint(model, epoch, optimizer, best_acc)
        
    print("Finished")
    return model

In [9]:
resnet18 = models.resnet18(pretrained=False)
num_features = resnet18.fc.in_features
num_classes = 7
resnet18.fc = nn.Linear(num_features, num_classes)

device = set_device()
resnet18 = resnet18.to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet18.parameters(), lr=0.01, momentum=0.9, weight_decay=0.003)

In [10]:
train_nn(resnet18, train_loader, test_loader, loss_func, optimizer, n_epochs=100)

Epoch: 1
Training dataset: 25812 out of 35700 correct. Accuracy: 72.303. Loss: 0.792
Testing dataset: 4116 out of 6300 correct. Accuracy: 65.333
Epoch: 2
Training dataset: 32944 out of 35700 correct. Accuracy: 92.28. Loss: 0.229
Testing dataset: 5935 out of 6300 correct. Accuracy: 94.206
Epoch: 3
Training dataset: 34054 out of 35700 correct. Accuracy: 95.389. Loss: 0.138
Testing dataset: 5970 out of 6300 correct. Accuracy: 94.762
Epoch: 4
Training dataset: 34308 out of 35700 correct. Accuracy: 96.101. Loss: 0.123
Testing dataset: 5974 out of 6300 correct. Accuracy: 94.825
Epoch: 5
Training dataset: 34622 out of 35700 correct. Accuracy: 96.98. Loss: 0.096
Testing dataset: 6142 out of 6300 correct. Accuracy: 97.492
Epoch: 6
Training dataset: 34463 out of 35700 correct. Accuracy: 96.535. Loss: 0.109
Testing dataset: 6138 out of 6300 correct. Accuracy: 97.429
Epoch: 7
Training dataset: 34574 out of 35700 correct. Accuracy: 96.846. Loss: 0.102
Testing dataset: 6153 out of 6300 correct. Accu

Testing dataset: 6187 out of 6300 correct. Accuracy: 98.206
Epoch: 58
Training dataset: 35119 out of 35700 correct. Accuracy: 98.373. Loss: 0.046
Testing dataset: 5164 out of 6300 correct. Accuracy: 81.968
Epoch: 59
Training dataset: 34613 out of 35700 correct. Accuracy: 96.955. Loss: 0.094
Testing dataset: 6212 out of 6300 correct. Accuracy: 98.603
Epoch: 60
Training dataset: 35200 out of 35700 correct. Accuracy: 98.599. Loss: 0.036
Testing dataset: 6188 out of 6300 correct. Accuracy: 98.222
Epoch: 61
Training dataset: 34426 out of 35700 correct. Accuracy: 96.431. Loss: 0.113
Testing dataset: 6218 out of 6300 correct. Accuracy: 98.698
Epoch: 62
Training dataset: 34826 out of 35700 correct. Accuracy: 97.552. Loss: 0.077
Testing dataset: 6101 out of 6300 correct. Accuracy: 96.841
Epoch: 63
Training dataset: 35114 out of 35700 correct. Accuracy: 98.359. Loss: 0.046
Testing dataset: 6216 out of 6300 correct. Accuracy: 98.667
Epoch: 64
Training dataset: 34769 out of 35700 correct. Accuracy

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [11]:
checkpoint = torch.load(f'{CHECKPOINT_PATH}\model_best_checkpoint.pth.tar')
print(f"Best Epoch: {checkpoint['epoch']}, Best Accuracy: {round(checkpoint['best accuracy'], 3)}")

Best Epoch: 96, Best Accuracy: 98.746


In [12]:
ONNX_SAVE_PATH = r"C:\Users\offco\Documents\Dev_Projects\BookCoverClassifier\Test_Run\Quantized Model\pytorch_resnet18.onnx"

resnet18 = models.resnet18(pretrained=False)
num_features = resnet18.fc.in_features
num_classes = 7
resnet18.fc = nn.Linear(num_features, num_classes)
resnet18.load_state_dict(checkpoint['model'])

dummy_input = torch.randn(1, 3, 256, 256, dtype=torch.float32)
torch.onnx.export(
    resnet18, 
    dummy_input, 
    ONNX_SAVE_PATH,
    verbose=False,
    export_params=True,
    opset_version=11,
    input_names = ['image'],
    output_names = ['pred']
)